In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This section imports the necessary libraries for data processing and deep learning.

In [ ]:
import numpy as np


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

We define the dataset path and set up image augmentation to improve model performance.


In [ ]:
data_dir = "/kaggle/input/bottle-cap-classification"  

img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2  
)

Load training and validation data

In [ ]:
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 615 images belonging to 1 classes.
Found 153 images belonging to 1 classes.

We define a Convolutional Neural Network (CNN) model with multiple layers to classify bottle caps.


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)), 
    MaxPooling2D(2,2),  
    Conv2D(64, (3,3), activation='relu'), 
    MaxPooling2D(2,2),  
    Flatten(),  
    Dense(128, activation='relu'),  
    Dropout(0.5), 
    Dense(1, activation='sigmoid') 
])


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
epochs = 10
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs
)

After training, we save the model and visualize its accuracy over epochs.


In [ ]:
model.save("bottle_cap_model.h5")


In [ ]:
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

We load the trained model and test it on a new image for classification.


In [ ]:
def predict_bottle_cap(image_path, model):
    img = image.load_img(image_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    return "Tilted Cap" if prediction > 0.5 else "Sealed Cap"

In [ ]:
loaded_model = tf.keras.models.load_model("bottle_cap_model.h5")

In [ ]:
import os

image_folder = "/kaggle/input/test-data" 
print("Files in directory:", os.listdir(image_folder))


In [ ]:
image_path = "/kaggle/input/test-data/Plastic_Bottle_Cap.JPG"  
result = predict_bottle_cap(image_path, loaded_model)
print("Prediction:", result)
